# 聊天代理执行者

在此示例中，我们将构建一个聊天执行器，该执行器使用以下函数调用
划痕。

## 设置¶

首先我们需要安装需要的包
```bash
yarn add langchain @langchain/openai @langchain/langgraph
```

接下来，我们需要为 OpenAI（我们将使用的 LLM）和 Tavily（我们将使用的 LLM）设置 API 密钥
我们将使用的搜索工具）
```bash
export OPENAI_API_KEY=
export TAVILY_API_KEY=
```

或者，我们可以设置 API 密钥
[LangSmith 追踪](https://smith.langchain.com/)，这将为我们提供
一流的可观测性。
```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=
```

## 设置工具

我们将首先定义我们想要使用的工具。对于这个简单的示例，我们将通过 Tavily 使用内置搜索工具。然而，创建自己的工具确实很容易 - 请参阅[此处](https://js.langchain.com/docs/how_to/custom_tools/)文档了解如何做到这一点。

定义我们的工具后，我们可以将它们包装在一个简单的 `ToolExecutor` 中。这是一个真正简单的
类接受 `ToolInvocation` 并调用该工具，返回输出。

ToolIncation 是具有 `tool` 和 `toolInput` 属性的任何类型。

In [ ]:
import { ToolExecutor } from "@langchain/langgraph/prebuilt";
import { TavilySearch } from "@langchain/tavily";

const tools = [new TavilySearch({ maxResults: 1 })];

const toolExecutor = new ToolExecutor({
  tools,
});

## 设置模型

现在我们需要加载我们想要使用的聊天模型。重要的是，这应该
满足两个条件：

1.它应该与消息一起使用。我们将以表格形式代表所有代理状态
消息，因此它需要能够与它们很好地配合。
2. 需要配合OpenAI函数调用。这意味着它应该是
OpenAI 模型或公开类似接口的模型。

注意：这些模型要求不是使用 LangGraph 的要求 - 它们
只是这个例子的要求。

In [5]:
import { ChatOpenAI } from "@langchain/openai";

// 我们将设置streaming=True，以便我们可以流式传输令牌
// 有关详细信息，请参阅流媒体部分。
const model = new ChatOpenAI({
  temperature: 0,
  streaming: true,
});

完成此操作后，我们应该确保模型知道它具有这些
可供调用的工具。我们可以通过将 LangChain 工具转换为
OpenAI函数调用的格式，然后将它们绑定到模型类。

In [6]:
import { convertToOpenAIFunction } from "@langchain/core/utils/function_calling";

const toolsAsOpenAIFunctions = tools.map((tool) =>
  convertToOpenAIFunction(tool)
);
const newModel = model.bind({
  functions: toolsAsOpenAIFunctions,
});

### 定义代理状态

`langgraph` 中图形的主要类型是 `StatefulGraph`。该图是
由传递到每个节点的状态对象参数化。每个节点
然后返回更新该状态的操作。这些操作可以设置
状态的特定属性（例如覆盖现有值）或添加到
现有的属性。通过注释状态来表示是设置还是添加
您构建图表所用的对象。

对于这个例子，我们将跟踪的状态只是一个消息列表。我们
希望每个节点只将消息添加到该列表中。为此，我们通过 `Annotation` 函数定义我们的状态，其中单个属性 `messages` 是 `BaseMessage` 的列表。我们的属性 `messages` 的值是另一个 `Annotation`，它有两个子属性：`reducer` 和 `default`。`reducer` 键必须是一个返回函数的工厂，该函数接受属性的当前值和要添加的新值，并返回属性的新值。

In [11]:
import { BaseMessage } from "@langchain/core/messages";
import { Annotation } from "@langchain/langgraph";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  })
})

## 定义节点

我们现在需要在图中定义一些不同的节点。在`langgraph`中，一个节点
可以是一个函数或一个
[可运行](https://js.langchain.com/docs/expression_language/)。有两个
为此我们需要的主要节点：

1. 代理：负责决定采取什么（如果有）行动。
2. 调用工具的函数：如果代理决定采取行动，该节点
然后将执行该操作。

我们还需要定义一些边。其中一些边缘可能是有条件的。
它们是有条件的原因是基于节点的输出，其中之一
可以采取多条路径。直到该节点才知道所采取的路径
运行（LLM 决定）。

1. 条件边缘：调用代理后，我们应该：如果
代理说要采取行动，那么调用工具的函数应该是
称为 b.如果代理说已经完成，那么就应该完成
2. 正常边缘：调用工具后，应始终返回到
代理决定下一步做什么

让我们定义节点以及一个函数来决定什么条件
边取。

In [12]:
import { FunctionMessage } from "@langchain/core/messages";
import { AgentAction } from "@langchain/core/agents";

// 定义判断是否继续的函数
const shouldContinue = (state: typeof AgentState.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1];
  // 如果没有函数调用，那么我们就完成了
  if (
    !("function_call" in lastMessage.additional_kwargs) ||
    !lastMessage.additional_kwargs.function_call
  ) {
    return "end";
  }
  // 否则，如果有，我们继续
  return "continue";
};

// 定义执行工具的函数
const _getAction = (state: typeof AgentState.State): AgentAction => {
  const { messages } = state;
  // 基于继续条件
  // 我们知道最后一条消息涉及函数调用
  const lastMessage = messages[messages.length - 1];
  if (!lastMessage) {
    throw new Error("No messages found.");
  }
  if (!lastMessage.additional_kwargs.function_call) {
    throw new Error("No function call found in message.");
  }
  // 我们从 function_call 构造一个 AgentAction
  return {
    tool: lastMessage.additional_kwargs.function_call.name,
    toolInput: JSON.stringify(
      lastMessage.additional_kwargs.function_call.arguments,
    ),
    log: "",
  };
};

// 定义调用模型的函数
const callModel = async (state: typeof AgentState.State) => {
  const { messages } = state;
  const response = await newModel.invoke(messages);
  // 我们返回一个列表，因为这将被添加到现有列表中
  return {
    messages: [response],
  };
};

const callTool = async (state: typeof AgentState.State) => {
  const action = _getAction(state);
  // 我们调用 tool_executor 并得到响应
  const response = await toolExecutor.invoke(action);
  // 我们使用响应来创建 FunctionMessage
  const functionMessage = new FunctionMessage({
    content: response,
    name: action.tool,
  });
  // 我们返回一个列表，因为这将被添加到现有列表中
  return { messages: [functionMessage] };
};

## 定义图表

我们现在可以将它们放在一起并定义图表！

In [13]:
import { END, START, StateGraph } from "@langchain/langgraph";

// 定义一个新图
const workflow = new StateGraph(AgentState)
  // 定义我们将在其之间循环的两个节点
  .addNode("agent", callModel)
  .addNode("action", callTool)
  // 将入口点设置为“agent”
  // 这意味着该节点是第一个被调用的节点
  .addEdge(START, "agent")
  // 我们现在添加一个条件边
  .addConditionalEdges(
    // 首先，我们定义起始节点。我们使用“代理”。
    // 这意味着这些是调用“agent”节点后获取的边。
    "agent",
    // 接下来，我们传入将确定接下来调用哪个节点的函数。
    shouldContinue,
    // 最后我们传入一个映射。
    // 键是字符串，值是其他节点。
    // END 是一个特殊的节点，标记图形应该结束。
    // 将会发生的事情是我们将调用“should_continue”，然后该输出
    // 将与此映射中的键进行匹配。
    // 根据匹配的节点，将调用该节点。
    {
      // 如果是“tools”，那么我们称之为工具节点。
      continue: "action",
      // 否则我们就结束了。
      end: END,
    },
  )
  // 我们现在添加从“tools”到“agent”的正常边缘。
  // 这意味着在调用“tools”之后，接下来调用“agent”节点。
  .addEdge("action", "agent");

// 最后，我们编译它！
// 这会将其编译成 LangChain Runnable，
// 这意味着您可以像使用任何其他可运行程序一样使用它
const app = workflow.compile();

## 使用它！

我们现在可以使用它了！现在这暴露了
[相同的接口](https://python.langchain.com/docs/expression_language/) 与所有
其他 LangChain 运行程序。

In [14]:
import { HumanMessage } from "@langchain/core/messages";

const inputs = {
  messages: [new HumanMessage("what is the weather in sf")],
};
await app.invoke(inputs);

{
  messages: [
    HumanMessage {
      "content": "what is the weather in sf",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessageChunk {
      "id": "chatcmpl-9y3695OyrZqhRmcbVkioOxXzjXp32",
      "content": "",
      "additional_kwargs": {
        "function_call": {
          "name": "tavily_search_results_json",
          "arguments": "{\"input\":\"weather in San Francisco\"}"
        }
      },
      "response_metadata": {
        "estimatedTokenUsage": {
          "promptTokens": 80,
          "completionTokens": 21,
          "totalTokens": 101
        },
        "prompt": 0,
        "completion": 0,
        "finish_reason": "function_call",
        "system_fingerprint": null
      },
      "tool_calls": [],
      "tool_call_chunks": [],
      "invalid_tool_calls": [],
      "usage_metadata": {
        "input_tokens": 79,
        "output_tokens": 21,
        "total_tokens": 100
      }
    },
    FunctionMessage {
      "content": "[{\"title\":\"We

这可能需要一点时间——它在幕后进行了一些调用。为了
为了开始看到一些中间结果，我们可以使用流式处理。
有关详细信息，请参阅下文。

## 流媒体

LangGraph 支持多种不同类型的流式传输。

### 流节点输出

使用 LangGraph 的好处之一是可以轻松地将输出流式传输为
它是由每个节点产生的。

在此示例中，我们将重新使用上面的 `inputs` 对象。

In [15]:
for await (const output of await app.stream(inputs)) {
  console.log("output", output);
  console.log("-----\n");
}

output {
  agent: {
    messages: [
      AIMessageChunk {
        "id": "chatcmpl-9y36G4P6hDihhZkOCW5T5bJWWNl0Z",
        "content": "",
        "additional_kwargs": {
          "function_call": {
            "name": "tavily_search_results_json",
            "arguments": "{\"input\":\"weather in San Francisco\"}"
          }
        },
        "response_metadata": {
          "estimatedTokenUsage": {
            "promptTokens": 80,
            "completionTokens": 21,
            "totalTokens": 101
          },
          "prompt": 0,
          "completion": 0,
          "finish_reason": "function_call",
          "system_fingerprint": null
        },
        "tool_calls": [],
        "tool_call_chunks": [],
        "invalid_tool_calls": [],
        "usage_metadata": {
          "input_tokens": 79,
          "output_tokens": 21,
          "total_tokens": 100
        }
      }
    ]
  }
}
-----

output {
  action: {
    messages: [
      FunctionMessage {
        "content": "[{\"title\":